In [ ]:
import pandas as pd
import numpy as np

## スタッフDB編集

In [ ]:
file_path = 'data/nyutai_data.csv'
df = pd.read_csv(file_path)

In [ ]:
dct = {
    'IDm/UID': "UID",
    '職員番号': "staff_number", 
    '漢字氏名': "kanji_name", 
    'カタカナ': "kana_name", 
    '所属': "dept", 
    '職種': "job", 
    '所属コード': "dept_code", 
    '区分コード': "class_code", 
    '鍵データ': "key_data",
    'カードデータ': "internal_card_data", 
    '発行日': "issue_date", 
    '修正日': "update_date",
}
df.rename(columns=dct, inplace=True)
df.dropna(subset=['UID'], how='all', axis=0, inplace=True)
df.set_index('UID', drop=True, inplace=True)

In [ ]:
l = []
for uid, name in df.kanji_name.to_dict().items():
    if type(name) == str:
        name = name.replace('　　', ' ')
        name = name.replace('　 ', ' ')
        name = name.replace(' 　', ' ')
        name = name.replace('　', ' ')
        name = name.split(" ")
    else:
        name = [name] # nanの対応

    length = len(name)
    if length >= 3: # 3つ以上の場合は、最初の1つを姓、残りを名に分ける
        name = [name[0], (" ").join(name[1:])]
    elif length == 1: # 1つの場合は、名を空欄にする
        name = name + [np.nan]
    l.append([uid, name[0], name[1]])

In [ ]:
df_kanji = pd.DataFrame(l, columns=['UID', 'kanji_last_name', 'kanji_first_name'])
df_kanji.set_index('UID', drop=True, inplace=True)

In [ ]:
l = []
for uid, name in df.kana_name.to_dict().items():
    if type(name) == str:
        name = name.replace('  ', ' ')
        name = name.split(" ")
    else:
        name = [name] # nanの対応

    length = len(name)
    if length >= 3: # 3つ以上の場合は、最初の1つを姓、残りを名に分ける
        name = [name[0], (" ").join(name[1:])]
    elif length == 1: # 1つの場合は、名を空欄にする
        name = name + [np.nan]
    l.append([uid, name[0], name[1]])

In [ ]:
df_kana = pd.DataFrame(l, columns=['UID', 'kana_last_name', 'kana_first_name'])
df_kana.set_index('UID', drop=True, inplace=True)

In [ ]:
df_merge = pd.merge(df, df_kanji, on='UID', how='left')
df_merge = pd.merge(df_merge, df_kana, on='UID', how='left')

In [ ]:
df_merge.drop(columns=['kanji_name', 'kana_name'], inplace=True)
df_merge = df_merge.reindex(columns=[
    'staff_number', 'kanji_last_name', 'kanji_first_name', 'kana_last_name', 'kana_first_name', 
    'dept', 'job', 'dept_code', 'class_code', 'key_data', 'internal_card_data', 'issue_date', 'update_date'
])

In [ ]:
df_merge.to_csv('data/nyutai_data_processed.csv', index=True, header=True)